In [ ]:
import os
import csv
import pandas as pd
import json
from tqdm import tqdm
import time



In [ ]:
import google.generativeai as genai
from google.colab import userdata
userdata.get('gemini_key')
API_KEY = userdata.get('gemini_key')
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.5-pro")

In [1]:
def build_prompt(transcript_text: str) -> str:
    return f"""

You are an expert tutor reviewer.The data you are given are limited by the following:

1.They are not diarized, You would have to know who the tutor is by looking at the utterance itself.
2.Lack of the problem the student is working on. The evidence of a student making an error might be from the reaction of the tutor afterwards.

Analyze the tutoring Audio transcripts below and score the tutor on the following 6 binary (0 or 1) dimensions.
For each dimension, return both a binary score (0 or 1) and a short evidence from the transcript under that dimension name.

RUBRIC DIMENSIONS (Return 0 or 1 for each)

1. reacting_to_errors:
    -0 = The student made a mistake and the tutor either gave the answer OR pointed out the error directly.
    -1 = The tutor responded to a math error by asking the student to explain their thinking OR prompting them to think again.
    -0 = The tutor does not react to any math error.


2. giving_praise

    -1 = Tutor Praises the student for their effort during the problem  (e.g.“good job. now,what about this one?”, “I like how you kept working on that question”)
    -0 = Tutor does not praise the student

3. determining_what_students_know

    -1 = Tutor asks open-ended questions to check what the student already knows on a problem (e.g., “How would you start?”)
    -0 = No attempt to check prior knowledge

4. affirming_correct_attempt

    -1 = Tutor affirms a correct response or student's correct reasoning (e.g., “Yes, that’s right!”)
    -0 = Tutor does not respond to a correct attempt to an explanation of the student's reasoning
    -0 = Tutor does not respond to a correct answer

5. asking_guiding_questions

    -1 = Tutor asks guiding questions (e.g., “What’s the next step?”, “What are we trying to solve?”)
    -0 = Tutor only gives instructions or answers without questions

6. prompting_explanation

    -1 = Tutor prompts student to explain their thinking (e.g., “Can you explain how you got that answer?”, "do you want to talk through what you're thinking?")
    -0 = No prompt to explain reasoning

Return your output STRICTLY in this JSON format:

```json
{{

  "reacting_to_errors": 0,
  "reacting_to_errors_evidence": "No evidence of a student making a math error",
  "giving_praise": 1,
  "giving_praise_evidence": "[03:00]: 'Nice job!'",
  "determining_what_students_know": 0,
  "determining_what_students_know_evidence": "No open-ended question to assess prior knowledge.",
  "affirming_correct_attempt": 1,
  "affirming_correct_attempt_evidence": "[05:01]: 'Yes, that’s exactly right.'",
  "asking_guiding_questions": 1,
  "asking_guiding_questions_evidence": "[06:12]: 'What should we do first here?'",
  "prompting_explanation": 0,
  "prompting_explanation_evidence": "No evidence the tutor asked the student to explain their thinking."

}}
Audio transcript:
\"\"\"
{transcript_text}
\"\"\"
"""


In [ ]:
ROOT_DIR = " "
OUTPUT_CSV = ""
BATCH_SIZE = 20
WAIT_TIME_BETWEEN_BATCHES = 5

fieldnames = [
    "file",
    "reacting_to_errors",
    "giving_praise",
    "determining_what_students_know",
    "affirming_correct_attempt",
    "asking_guiding_questions",
    "prompting_explanation",
   

]

# Track already processed files
completed_files = set()
if os.path.exists(OUTPUT_CSV):
    df_existing = pd.read_csv(OUTPUT_CSV)
    completed_files = set(df_existing["file"].tolist())

# Collect new .vtt files
all_vtt_files = [
    f for f in os.listdir(ROOT_DIR)
    if f.endswith(".vtt") and f not in completed_files
]

def process_file(filename):
    file_path = os.path.join(ROOT_DIR, filename)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            transcript_text = f.read()

        prompt = build_prompt(transcript_text)
        response = model.generate_content(prompt)
        content = response.text.strip()

        if content.startswith("```json"):
            content = content.replace("```json", "").replace("```", "").strip()

        scores = json.loads(content)

    #    model output with evidence
        print(f"\n📘 {filename} \n{json.dumps(scores, indent=2)}\n")

        
        row = {"file": filename}
        for dim in fieldnames[1:]:
            row[dim] = scores.get(dim, 0)

        return row

    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return None

# batch processing
for batch_start in range(0, len(all_vtt_files), BATCH_SIZE):
    batch_files = all_vtt_files[batch_start: batch_start + BATCH_SIZE]
    print(f"\n Processing batch {batch_start // BATCH_SIZE + 1} ({len(batch_files)} files)")

    batch_results = []
    for filename in tqdm(batch_files):
        result = process_file(filename)
        if result:
            batch_results.append(result)

    # Save just the scores
    if batch_results:
        pd.DataFrame(batch_results).to_csv(
            OUTPUT_CSV, mode="a", header=not os.path.exists(OUTPUT_CSV), index=False
        )
        print(f"🍀Batch saved: {len(batch_results)} results")
    else:
        print("No results in this batch (all failed?)")

    print(f"Waiting {WAIT_TIME_BETWEEN_BATCHES} seconds before next batch...")
    time.sleep(WAIT_TIME_BETWEEN_BATCHES)
